In [1]:
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.linear_model import LogisticRegression
from gensim.parsing.preprocessing import STOPWORDS
import lightgbm as lgb
from catboost import CatBoostRegressor
import tensorflow_hub as hub
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout
from keras import layers
%matplotlib inline

In [2]:
geo_train = pd.read_csv('csv/test_con_geo.csv')
geo_test = pd.read_csv('csv/train_con_geo.csv')
test = pd.read_csv('csv/results_trees_nets_tf_keyw_test.csv')
train = pd.read_csv('csv/results_trees_nets_tf_keyw_train.csv')
test_red = pd.read_csv('eze_pulido/submission3000float.csv')
train_red = pd.read_csv('eze_pulido/submission3000_train_float.csv')
train_target = pd.read_csv('csv/train.csv')

In [3]:
test['best'] = test_red['target'].copy()
train['best'] = train_red['target'].copy()

In [4]:
geo_train.sample()

,Unnamed: 0,id,keyword,location,text,loc_nula,loc_valida,coordenadas,latitud,longitud
2856,2856,9473,terrorism,"Washington, DC",New US counter #terrorism program urges #Musli...,0,1,"(38.8949855, -77.0365708)",38.894985,-77.036571


In [5]:
train['latitud']  = geo_train['latitud']
train['longitud'] = geo_train['longitud']
test['latitud']  = geo_test['latitud']
test['longitud'] = geo_test['longitud']


In [6]:
train.sample()

,id,word2vec_tree,elmo_tree,glove_tree,bert_tree,elmo_net,bert_net,w2v_net,glove_net,tf_uni,tf_bi,tf_tri,keyword_veracity,best,latitud,longitud
2847,4093,0.532221,0.422672,0.412649,0.464573,0.034943,0.037549,0.465988,0.323211,0.184584,0.268707,0.302611,0.722222,0.053023,52.795479,-0.54024


In [7]:
test.sample()

,id,word2vec_tree,elmo_tree,glove_tree,bert_tree,elmo_net,bert_net,w2v_net,glove_net,tf_uni,tf_bi,tf_tri,keyword_veracity,best,latitud,longitud
533,1743,0.624515,0.506749,0.392414,0.330172,0.537293,0.291364,0.629312,0.511867,0.488266,0.648541,0.572929,0.685714,0.78509,NaN,NaN


In [8]:
del test ['id']
del train ['id']
train  = train.fillna( 0.00)
test = test.fillna ( 0.00 )

# Árboles

In [9]:
X, y = train, train_target['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=18)

In [10]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 5)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.9149159663865546
Precision Score : 0.9288486416558862
Recall Score : 0.8703030303030304
F1 Score : 0.8986232790988735


# Redes

In [11]:
model = Sequential()
model.add(layers.Dense(7, input_dim=train.shape[1], activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, input_dim=train.shape[1], activation='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 112       
_________________________________________________________________
dropout (Dropout)            (None, 7)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 8         
Total params: 120
Trainable params: 120
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(X_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=5)

Epoch 1/5
1142/1142 [==============================] - 2s 1ms/step - loss: 0.6870 - accuracy: 0.7669 - val_loss: 0.3878 - val_accuracy: 0.8745
Epoch 2/5
1142/1142 [==============================] - 1s 1ms/step - loss: 0.4664 - accuracy: 0.8439 - val_loss: 0.2682 - val_accuracy: 0.8871
Epoch 3/5
1142/1142 [==============================] - 1s 1ms/step - loss: 0.3407 - accuracy: 0.8672 - val_loss: 0.2231 - val_accuracy: 0.8981
Epoch 4/5
1142/1142 [==============================] - 1s 1ms/step - loss: 0.3085 - accuracy: 0.8784 - val_loss: 0.2030 - val_accuracy: 0.9044
Epoch 5/5
1142/1142 [==============================] - 1s 1ms/step - loss: 0.2782 - accuracy: 0.8840 - val_loss: 0.1970 - val_accuracy: 0.9028


In [14]:
preds = model.predict(X_test)
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.9028361344537815
Precision Score : 0.9481792717086834
Recall Score : 0.8206060606060606
F1 Score : 0.8797920727745289


In [18]:
preds = model.predict(test) #me quedo con la de la red
len(preds)

3263

In [29]:
real_test = pd.read_csv('csv/test.csv')
real_test['target'] = preds.round().astype(int)
real_test.set_index('id', inplace=True)
real_test = real_test['target']
real_test.to_csv('submission_red_con_geo.csv')

In [30]:
real_test

id
0        1
2        1
3        1
9        1
11       1
        ..
10861    1
10865    1
10868    0
10874    1
10875    0
Name: target, Length: 3263, dtype: int64